In [1]:
import pandas as pd

import matplotlib.pyplot as plt

from collections import defaultdict

from model_io.Base import Base
from model_io.Devices import Devices
from model_io.Io import Io

from sqlalchemy import create_engine, text, func
from sqlalchemy.orm import sessionmaker
from sqlalchemy.pool import NullPool

from IPython.display import display

DB='postgresql+psycopg2:///ucnstudy_hostview_data'

engine = create_engine(DB, echo=False, poolclass=NullPool)
Base.metadata.bind = engine
Session = sessionmaker(bind=engine)

In [3]:
ses = Session()
devices = ses.query(Devices)

for device in devices:
    #select only users from ucnstudy
    if device.id == 5 or device.id == 6 or device.id == 8 or device.id == 11 or device.id == 12:
        print (device.device_id + '===============')

        sql_beg_day = text('SELECT distinct session_id, io.logged_at, io.device \
        FROM io join \
        (SELECT DATE(logged_at) as date_entered, MIN(logged_at) as min_time \
        FROM io \
        WHERE session_id =:dev_id and extract (hour from logged_at) > 3 \
        GROUP BY date(logged_at)) AS grp ON grp.min_time = io.logged_at order by io.logged_at;').bindparams(dev_id = device.id)
        result_beg_day = ses.execute(sql_beg_day)
            
        sql_end_day = text('SELECT distinct session_id, io.logged_at, io.device \
        FROM io join \
        (SELECT DATE(logged_at) as date_entered, MAX(logged_at) as max_time \
        FROM io \
        WHERE session_id =:dev_id \
        GROUP BY date(logged_at)) AS grp ON grp.max_time = io.logged_at order by io.logged_at;').bindparams(dev_id = device.id)
        result_end_day = ses.execute(sql_end_day)

        sql_beg_day_nolimit = text('SELECT distinct session_id, io.logged_at, io.device \
        FROM io join \
        (SELECT DATE(logged_at) as date_entered, MIN(logged_at) as min_time \
        FROM io \
        WHERE session_id =:dev_id \
        GROUP BY date(logged_at)) AS grp ON grp.min_time = io.logged_at order by io.logged_at;').bindparams(dev_id= device.id)
        result_beg_day_nolimit = ses.execute(sql_beg_day_nolimit)
            
        #organize data
        info_end = defaultdict(list)
        for row in result_end_day:
            info_end['devid'].append(row[0])
            info_end['ts_end'].append(row[1])
            info_end['interaction'].append(row[2])

        info_beg = defaultdict(list)
        for row in result_beg_day:
            info_beg['devid'].append(row[0])
            info_beg['ts_start'].append(row[1])
            info_beg['interaction'].append(row[2])  
                
        #add days that only have value before 3 am
        for row in result_beg_day_nolimit:
            timst = row[1]
            in_list = False
            for dt in info_beg['ts_start']:
                if dt.date() == timst.date():
                    in_list = True
            if in_list == False:
                #insert in the correct position
                cont = 0
                for dat in info_beg['ts_start']:
                    if timst.date() > dat.date():
                        cont = cont + 1
                        info_beg['ts_start'].insert(cont, timst)
                        info_beg['devid'].insert(cont, row[0])
                        info_beg['interaction'].insert(cont, row[2])
                    
        df_beg = pd.DataFrame(info_beg)
        display(df_beg)
        df_end = pd.DataFrame(info_end)
        display(df_end)
        
        
        #creates table per day for beginning
        info_week = defaultdict(list)
        if (info_beg['ts_start']):
            cont = 0
            for timst in info_beg['ts_start']:
                day = timst
                weekday = day.strftime('%A')
                info_week[weekday].append(day)
                info_week[weekday+'interaction'].append(info_beg['interaction'][cont])
                cont = cont + 1
         
        print (device.device_id + '===============')
        days_str = {'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday','Saturday', 'Sunday'}
        for name in days_str:
            df_col = defaultdict(list)
            df_col[name+' start'] = info_week[name]
            df_col['interaction'] = info_week[name+'interaction']
            df_week = pd.DataFrame(df_col)
            display(df_week)
            
                    #creates table per day for ending
        info_week = defaultdict(list)
        if (info_end['ts_end']):
            cont = 0
            for timst in info_end['ts_end']:
                day = timst
                weekday = day.strftime('%A')
                info_week[weekday].append(day)
                info_week[weekday+'interaction'].append(info_end['interaction'][cont])
                cont = cont + 1
             
        print (device.device_id + '===============')
        days_str = {'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday','Saturday', 'Sunday'}
        for name in days_str:
            df_col = defaultdict(list)
            df_col[name+' end'] = info_week[name] 
            df_col['interaction'] = info_week[name+'interaction']
            df_week = pd.DataFrame(df_col)
            display(df_week)

kemianny.mainlaptop===============


,devid,interaction,ts_start
0,5,3,2015-11-11 18:01:41.322
1,5,3,2015-11-13 10:42:44.273
2,5,4,2015-11-16 11:18:40.161
3,5,3,2015-11-17 11:08:40.846
4,5,4,2015-11-20 11:04:55.844
5,5,4,2015-11-21 13:39:23.411
6,5,4,2015-11-22 12:22:12.669
7,5,4,2015-11-23 13:09:47.428
8,5,3,2015-11-24 13:24:36.921
9,5,4,2015-11-25 14:30:50.615


,devid,interaction,ts_end
0,5,3,2015-11-11 18:34:36.603
1,5,3,2015-11-13 15:05:17.405
2,5,3,2015-11-16 15:32:33.088
3,5,1,2015-11-17 13:20:30.740
4,5,1,2015-11-20 16:15:26.665
5,5,3,2015-11-21 22:24:34.246
6,5,4,2015-11-22 22:50:05.022
7,5,4,2015-11-23 18:07:49.617
8,5,3,2015-11-24 15:50:13.527
9,5,3,2015-11-25 22:02:41.005


kemianny.mainlaptop===============


,Monday start,interaction
0,2015-11-16 11:18:40.161,4
1,2015-11-23 13:09:47.428,4


...

,Friday end,interaction
0,2015-11-13 23:24:17.985,4
1,2015-11-20 21:58:18.854,3


,Wednesday end,interaction
0,2015-11-11 15:49:49.677,3
1,2015-11-18 23:04:43.193,3
2,2015-11-25 23:58:08.959,3


,Thursday end,interaction
0,2015-11-12 22:19:06.758,3
1,2015-11-19 23:37:56.787,3
2,2015-11-26 17:21:33.594,3


,Sunday end,interaction
0,2015-11-15 22:00:03.229,3
1,2015-11-22 23:59:59.364,3


,Saturday end,interaction
0,2015-11-14 23:58:51.579,3
1,2015-11-21 20:50:52.640,3
